In [1]:
from keras.models import load_model
from skimage.io import imread

import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
image_path = 'image_data/test_v2/' # path to images
model_path = 'U-Net.h5' # path to model
weight_path = 'model_weights.h5' # path to model weights

normalize = 1/255.0 # normalize the image to a pixel value [0, 1]
BATCH_SIZE = 4 # number of images for prediction per iteration
image_shape = (768, 768)

os.environ["CUDA_VISIBLE_DEVICES"]="1"; # if available several GPU selects a second

In [3]:
model = load_model(model_path) # loading model
model.load_weights(weight_path) # loading weights into network

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/yura/anaconda3/envs/torchreid/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def rle_encode(img):
    """
    function for encoding model result in submission file by run-lenght encoding
    """
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0]
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [5]:
images = sorted(os.listdir(image_path))

# function for preparing images into model

def get_image(image):
    img = os.path.join(image_path, image)
    img = imread(img)
    img = np.expand_dims(img, 0)
    return img/255.0

In [6]:
# the model produces a blurred result 
# with a frame around the sides and top, so I zero it

def nullable_padding(img):
    img[-1] = 0
    img[:, 0] = 0
    img[:, -1] = 0
    return img

In [7]:
# creation of submission file

df = pd.DataFrame(columns=['ImageId', 'EncodedPixels'])
for i, ImageId in enumerate(images):
    img = get_image(ImageId)
    y = nullable_padding(model.predict(img).squeeze().round())
    EncodedPixels = rle_encode(y)
    df = df.append({'ImageId':ImageId, 'EncodedPixels': EncodedPixels}, ignore_index=True)
    
df.to_csv('sample_submission_v2.csv')

In [8]:
df

,ImageId,EncodedPixels
0,00002bd58.jpg,
1,00015efb6.jpg,
2,00023d5fc.jpg,
3,000367c13.jpg,10138 3 10177 2 10904 4 10944 4 11668 8 11710 ...
4,0008ca6e9.jpg,
5,000a4635f.jpg,
6,000bd9dbf.jpg,418499 3 419264 7 420029 11 420796 13 421564 1...
7,000f7d875.jpg,
8,0010551d9.jpg,173836 7 174601 13 175365 21 176130 27 176895 ...
9,001839c6f.jpg,272597 2 273362 7 274128 12 274895 14 275662 1...
